In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from darts.utils.likelihood_models import QuantileRegression
from torchmetrics import MeanSquaredError
from torch.nn import MSELoss
import sys
sys.path.append('../Helper/')
from dataPreprocessing import rank_features_ccf, get_untransformed_exog
from hyperparameters import tune_hyperparameters, load_best_hyperparameters, save_best_hyperparameters
import numpy as np
import torch
import optuna
from sklearn.metrics import mean_squared_error
from torch.optim import AdamW

In [2]:

data = pd.read_csv('../../Data/Train/train1990s.csv')

data= get_untransformed_exog(data)


In [3]:
NUM_SAMPLES=data.shape[0]
INPUT_SIZE=12
VALID_SIZE=3
HORIZON=1
valid_metric= MeanSquaredError()

In [4]:
df= data.copy()
df['observation_date'] = pd.to_datetime(df['observation_date'], format='%m/%Y')
df.set_index('observation_date', inplace=True)
ranked_exog_cols = rank_features_ccf(df,targetCol='fred_PCEPI')

In [5]:
def load_prediction(input_size, df_exog,target_series,exog_scaler, target_scaler, pred_date):

    X=TimeSeries.from_dataframe(df_exog.loc[pred_date-pd.DateOffset(months=input_size):pred_date-pd.DateOffset(months=1),:])
    y_past= TimeSeries.from_series(target_series.loc[pred_date-pd.DateOffset(months=input_size):pred_date-pd.DateOffset(months=1)])

    return exog_scaler.transform(X), target_scaler.transform(y_past)

def train_valid_split_darts(exog_df, target_series, valid_size,input_size):
    train_target= TimeSeries.from_series(target_series.iloc[:-valid_size])
    valid_target= TimeSeries.from_series(target_series.iloc[-valid_size-input_size:])

    train_exog= TimeSeries.from_dataframe(exog_df.iloc[:-valid_size,:])
    valid_exog= TimeSeries.from_dataframe(exog_df.iloc[-valid_size-input_size:,:])

    return train_target, valid_target, train_exog, valid_exog

In [ ]:
ADD PRUNING
def objective(trial):

    # Optuna suggestions:
    num_exog= trial.suggest_int("num_exog",1,df.shape[0],step=1)
    loss_fn_type= trial.suggest_categorical("loss_fn", ["QuantileRegression", "MSE"])

    if loss_fn_type=="QuantileRegression":
        likelihood= QuantileRegression((0.25,0.5,0.75))
        loss_fn=None
    else:
        likelihood=None
        loss_fn=torch.nn.MSELoss()
    lr= trial.suggest_float("lr",1e-5,1e-1,log=True)
    hidden_size=trial.suggest_int("hidden_size",1,32,step=1)
    lstm_layers=trial.suggest_int("lstm_layers",1,32,step=1)
    num_attention_heads=trial.suggest_int("num_attention_heads",1,8,step=1)


    # Modify data:
    df_exog=df[ranked_exog_cols[:num_exog]]
    
    train_target, valid_target, train_exog, valid_exog = train_valid_split_darts(df_exog,df['fred_PCEPI'],VALID_SIZE,INPUT_SIZE)

    target_scaler = Scaler()
    exog_scaler = Scaler()

    train_target_scaled = target_scaler.fit_transform(train_target)
    train_exog_scaled = exog_scaler.fit_transform(train_exog)

    valid_target_scaled = target_scaler.transform(valid_target)
    valid_exog_scaled = exog_scaler.transform(valid_exog)

    #Callbacks:
    early_stopper = EarlyStopping(
        monitor="val_MeanSquaredError",
        patience=25,
        min_delta=1e-4,
        mode="min",
        verbose=True
    )

    #Define model:
    model = TFTModel(
        input_chunk_length=INPUT_SIZE,
        output_chunk_length=HORIZON,
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        dropout=0.1,
        optimizer_cls=AdamW,
        n_epochs=100,
        random_state=42,
        likelihood=likelihood,
        optimizer_kwargs={
            "lr":lr,
            'betas':(0.9, 0.999), 
            'eps':1e-08, 
            'weight_decay':0.01, 
            'amsgrad':False,
            'maximize':False, 
            'foreach':None, 
            'capturable':False, 
            'differentiable':False, 
            'fused':None
        },
        pl_trainer_kwargs={
            "accelerator":"gpu",
            "devices": -1,
            "callbacks": [early_stopper]
        },
        loss_fn=loss_fn,
        add_encoders={
            "cyclic": {"future": ["month", "quarter"]}
        },
        save_checkpoints=True,
        torch_metrics=valid_metric,
        model_name='TFT_optuna',
        force_reset=True
    )


    model.fit(
        series=train_target_scaled,
        past_covariates=train_exog_scaled,
        val_series=valid_target_scaled,
        val_past_covariates=valid_exog_scaled,
        verbose=False,
    )

    model.load_from_checkpoint(model_name='TFT_optuna',best=True)

    valid_scores=[]
    ground_truth=[]

    for i in range(1,VALID_SIZE +1):
        x,y=load_prediction(INPUT_SIZE,df_exog,df['fred_PCEPI'],exog_scaler,target_scaler,df.index[-i])
        if isinstance(likelihood,QuantileRegression):
            # If the model is probabilistic, take 100 samples and average them
            preds_=[]
            for i in range(0,15):
                preds_.append(target_scaler.inverse_transform(model.predict(n=HORIZON,series=y,past_covariates=x,verbose=False)).values().reshape(-1))
            
            valid_scores.append(np.mean(preds_))

        else:
            valid_scores.append(target_scaler.inverse_transform(model.predict(n=HORIZON,series=y,past_covariates=x,verbose=False)).values().reshape(-1))
        
        ground_truth.append(df.loc[df.index[-i],'fred_PCEPI'])
    

    mse_score=mean_squared_error(np.asarray(valid_scores),np.asarray(ground_truth))
    
    return mse_score

In [7]:
best_hyper_params=tune_hyperparameters(objective, n_trials=5)
print(best_hyper_params)
save_best_hyperparameters(best_hyper_params)

loaded_hyper_params= load_best_hyperparameters()
print(loaded_hyper_params)

[I 2025-04-15 18:24:47,203] A new study created in memory with name: no-name-c447a4f2-1f49-40ca-8fea-ce59b74130e6
2025-04-15 18:24:47,637 - INFO - Train dataset contains 393 samples.
2025-04-15 18:24:47,991 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('train_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
Metric val_MeanSquaredError improved

2025-04-15 18:38:26,103 - INFO - Train dataset contains 393 samples.
2025-04-15 18:38:26,237 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('train_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
Metric val_MeanSquaredError improved. New best score: 0.534
Metric val_MeanSquaredError improved by 0.266 >= min_delta = 0.0001. New best score: 0.268

2025-04-15 18:44:20,236 - INFO - Train dataset contains 393 samples.
2025-04-15 18:44:20,372 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('train_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
Metric val_MeanSquaredError improved. New best score: 0.033
Metric val_MeanSquaredError improved by 0.005 >= min_delta = 0.0001. New best score: 0.028

2025-04-15 18:47:42,999 - INFO - Train dataset contains 393 samples.
2025-04-15 18:47:43,133 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('train_MeanSquaredError', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
Metric val_MeanSquaredError improved. New best score: 2.127
Metric val_MeanSquaredError improved by 1.195 >= min_delta = 0.0001. New best score: 0.932

NameError: name 'exit' is not defined